The goal here is to simulate a realistic GPS signal, including transmitter and receiver clock errors, as well as signal propagation delay based on satellite and receiver positions. The code snippets below show how to set up the simulation parameters, retrieve satellite positions from SP3 data, and compute the necessary delays and clock errors.

We will not include relativistic effects or atmospheric delays in this simplified model.

In [ ]:
import os
import utils
from datetime import datetime, timedelta
import gnss_tools.orbits as orbits
import gnss_tools.time as time
import gnss_tools.coords as coords
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Create local-data directory for storing downloaded data files
data_dir = os.path.join(os.path.dirname(os.path.dirname(utils.__file__)), "local-data")
os.makedirs(data_dir, exist_ok=True)

In [ ]:
# simulate static RX position
rx_pos_geo = np.array((-118.57707656024262, 34.139012915514925, 361.0))  # lon, lat, alt in meters
rx_pos_ecf = coords.geo2ecf(rx_pos_geo)

In [ ]:
# Get GNSS positions using SP3

sim_start_dt = datetime(2024, 5, 10)
sim_end_dt = sim_start_dt + timedelta(days=1)
sim_start_gpst = time.convert_datetime_to_gps_seconds(sim_start_dt)
sim_end_gpst = time.convert_datetime_to_gps_seconds(sim_end_dt)

sp3_arrays = orbits.download_and_parse_sp3_data(sim_start_gpst, sim_end_gpst, data_dir, True)
sat_sp3_sky_coords = {}
for sat_id in sp3_arrays.position.keys():
    sat_pos_ecf = sp3_arrays.position[sat_id]
    sat_sky_coords = coords.ecf2sky(
        rx_pos_ecf, sat_pos_ecf
    )
    sat_sp3_sky_coords[sat_id] = sat_sky_coords
# sat_id = "G01"
# sat_pos_ecf = sp3_arrays.position[sat_id]
# sat_pos_interp_ecf = orbits.compute_array_lagrange_interpolation(
#     geom_epochs_gpst - sim_start_gpst, sp3_arrays.epochs - sim_start_gpst, sat_pos_ecf, 5
# )
# geometric_range = np.linalg.norm(sat_pos_interp_ecf - rx_pos_ecf[None, :], axis=1)  # in meters
# speed_of_light = 299792458.0  # in meters per second
# propagation_delay = geometric_range / speed_of_light  # in seconds

In [ ]:
# Make sky plot of all satellites
# Check for elevation > 0 deg
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection="polar")
ax.set_theta_zero_location("N")
gnss_colors = {
    "G": "blue",
    "R": "green",
    "B": "red",
    "E": "orange",
}
for sat_id, sky_coords in sat_sp3_sky_coords.items():
    elevation = sky_coords[:, 1]
    azimuth = sky_coords[:, 0]
    mask = elevation > 0.0
    svid_sys = sat_id[0]
    if svid_sys not in gnss_colors:
        continue
    color = gnss_colors[svid_sys]
    ax.scatter(
        np.radians(azimuth[mask]),
        90.0 - elevation[mask],
        color=color,
        s=10,
    )
rticks = [0, 20, 40, 60, 80]
ax.set_yticks(rticks)
ax.set_yticklabels([str(90 - rtick) for rtick in rticks])
plt.show()

In [ ]:
samp_rate = 5e6  # 5 MHz
sim_duration = 1.0  # seconds
t = np.arange(0, sim_duration, 1 / samp_rate)